In [6]:
#爬歌手歌名
import requests
import pandas as pd
from bs4 import BeautifulSoup
def billboard_hot_100(year):
    url = requests.get('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_'+str(year))
    soup = BeautifulSoup(url.text)
    df=pd.DataFrame({'Rank':[],'Title':[],'Artist':[]})
    row=[]
    summary_soup = soup.find_all('table',class_="wikitable sortable")
    for i in range(0,len(summary_soup)):
        for j,entry in enumerate(summary_soup[i].find_all("td")):
            row.append(entry.text)
            if (j+1)%3 == 0:
                df.loc[len(df.index)]=row
                row = []
            
    df=df.applymap(lambda x: x.replace('"', ''))
    df=df.applymap(lambda x: x.replace('\n', ''))
    df['Artist']=df['Artist'].replace(regex='featuring', value='feat')
    df.insert(0,'Year',value=year)
    return df


frames=[]
for i in range(2012,2023):
    frames.append(billboard_hot_100(i))
result = pd.concat(frames)

result.to_csv(r'/Users/yuhong/Downloads/billboard_hot_100_2012to2022.csv', index=False)

In [4]:
#爬歌词
import lyricsgenius
import pandas as pd
def get_lyrics(title, artist):
    # set up API key and create genius client
    api_key = 'iQ-nun6SmRRljnmQYzJVEK96sh2DZuIbC4EcitT_fR45Wjy1bnpInQzsGNWo0UQo'  # replace with your own API key
    genius = lyricsgenius.Genius(api_key, timeout=50)

    # search for song lyrics
    song = genius.search_song(title, artist)

    # check if lyrics are found, and return them if so
    if song:
        return song.lyrics
    else:
        return "Sorry, couldn't find lyrics for this song."

df=pd.read_csv("billboard_hot_100_2012to2022.csv")
df['Lyrics']=''    
for i in range(0,1100):
    df['Lyrics'][i]=get_lyrics(df['Title'][i],df['Artist'][i])
df.to_csv(r'/Users/yuhong/Downloads/billboard_hot_100_2012to2022_with_lyrics.csv', index=False)